In [1]:
%matplotlib inline
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

import warnings
warnings.filterwarnings("ignore")
import sys
import datetime
import scipy as sp
import statsmodels.stats.api as sms
import statsmodels.api as sm
from patsy import dmatrix
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
import utils.preprocessing as pp 
import utils.correlation as cr
import utils.statsmodel_helper as st

# df_macro = pd.read_csv('./data/macro.csv', parse_dates=['timestamp'])
df_train = pd.read_csv('./data/train_imput_regression.csv', index_col=0)
df_test = pd.read_csv('./data/test_imput_regression.csv', index_col=0)

min_corr = 0.3

In [2]:
def pick_highly_correlated_features(df, columns, min_corr):
    pairs = []
    for col in columns:
        if not np.issubdtype(df[col].dtype, np.number):continue
        corrs = [(col, c, abs(df[col].corr(df[c]))) for c in df.columns.values.tolist() if c != col]
        corrs.sort(key=lambda item: item[2], reverse=True)
        for item in corrs:
            if item[2] > min_corr:
                pairs.append(item)
            else:
                break
    return pd.DataFrame(pairs, columns=['missing_col', 'highest corr with', 'corr'])

def pick_highly_correlated_IVs(df, target_col, min_corr, min_unique_values = 0):
    if not np.issubdtype(df[target_col].dtype, np.number):
        Exception('{}은 numeric data가 아닙니다.'.format(target_col))
    # if len(df[col].value_counts().index) < min_unique_values:
    #     Exception('{}로 상관관계를 계산하기에는 유니크한 값이 너무 작습니다.'.format(col))

    corrs = []
    for col in df._get_numeric_data().drop(target_col, axis=1).columns:
        if len(df[col].value_counts().index) < min_unique_values: continue
        corr = abs(df[target_col].corr(df[col]))
        if corr > min_corr:
            corrs.append((col, corr))
    
    return corrs 

In [4]:
columns = list(df_train._get_numeric_data().columns)

In [5]:
df_train[columns].astype(int).tail()

,usdrub,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,state,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25_raion,shopping_centers_raion,office_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,big_road2_km,railroad_km,zd_vokzaly_avto_km,bus_terminal_avto_km,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,cafe_avg_price_2000,cafe_count_2000_na_price,cafe_count_2000_price_500,cafe_count_2000_price_1000,cafe_count_2000_price_1500,cafe_count_2000_price_2500,cafe_count_2000_price_4000,cafe_count_2000_price_high,big_church_count_2000,church_count_2000,mosque_count_2000,leisure_count_2000,sport_count_2000,market_count_2000,green_part_3000,prom_part_3000,office_count_3000,office_s

In [ ]:
pairs = []

for col in columns:
    if not np.issubdtype(df_train[col].dtype, np.number):continue
    corrs = [(col, c, abs(df_train[col].corr(df_train[c]))) for c in df_train.columns.values.tolist() if c != col]
    corrs.sort(key=lambda item: item[2], reverse=True)
    for item in corrs:
        if item[2] > 0.3:
            pairs.append(item)
        else:
            break
df = pd.DataFrame(pairs, column=['missing_col', 'highest corr with', 'corr'])

In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30404 entries, 1 to 30473
Columns: 296 entries, usdrub to avg_price_sub_area
dtypes: float64(129), int64(152), object(15)
memory usage: 68.9+ MB


In [15]:
pairs = []
for col in columns:
    if not np.issubdtype(df_train[col].dtype, np.number):continue
    corrs = [(col, c, abs(df_train[col].corr(df_train[c]))) for c in df_train.columns.values.tolist() if c != col]
    corrs.sort(key=lambda item: item[2], reverse=True)
    for item in corrs:
        if item[2] > min_corr:
            pairs.append(item)
        else:
            break
df = pd.DataFrame(pairs, columns=['missing_col', 'highest corr with', 'corr'])

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [6]:
pick_highly_correlated_features(df_train, columns, 0.3)

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
pick_highly_correlated_IVs(df_train, 'price_doc', 0.3)